In [1]:
import experiment_manager as xpm
from pathlib import Path

import os
from os.path import join

import pandas as pd

import altair as alt
currpath=os.getcwd()

In [2]:
#construct absolute path to result directory

result_path = join(Path(currpath).parents[0], 'results', 'noniid')

list_of_experiments = xpm.get_list_of_experiments(result_path)
data = list_of_experiments[14].results
print()
hyp = data['hyperparameters']
del data['hyperparameters']

Loaded  15  Results from  /home/tim/HHI/fl_distill/results/noniid

Hyperparameters: 
 {'dataset': ['cifar10'], 'distill_dataset': ['stl10'], 'net': ['vgg11sb', 'vgg11s'], 'n_clients': [20], 'classes_per_client': [0.1], 'balancedness': [1.0], 'communication_rounds': [20], 'participation_rate': [0.4], 'local_epochs': [20], 'distill_epochs': [10], 'n_distill': [100000], 'warmup_type': ['constant', 'tanh'], 'batch_size': [128], 'local_data_percentage': [0.5, 0.05], 'distill_weight': [10, 1, 0.1, 20], 'aggregation_mode': ['FD'], 'distill_mode': ['regular'], 'distill_phase': ['clients'], 'only_linear': [False], 'pretrained': [None], 'save_model': [None], 'log_frequency': [-100], 'log_path': ['noniid/'], 'job_id': [[141208], [141427], [141424]], 'finished': [True, False], 'log_id': [14394, 11049, 43142, 95295, 34123, 95887, 28901, 78373, 45378, 55302, 68021, 43577, 63922, 22513, 34502]}

Tracked Variables: 
 ['hyperparameters', 'server_val_accuracy', 'distill_loss', 'distill_acc', 'distill_ep

In [3]:
def generate_plot(data, hyp):
    data['server_val_accuracy'] = data['server_val_accuracy']
    count = data['distill_acc'].shape[1] if data['distill_acc'].ndim == 2 else 1
    distill_acc = pd.DataFrame(data['distill_acc'], columns=[f'client{i}'for i in range(count)])
    distill_acc.index.name = "round"
    distill_acc = distill_acc.reset_index().melt('round')


    base = alt.Chart(distill_acc).transform_calculate(
        line="'Average'",
        shade="'Sigma1 confidence interval'",
    ).properties(
        title={
            'text': 'Rounds vs. Performance for One Step Distillation',
            'subtitle': f"Net: {hyp['net']} - Phase: {hyp['distill_phase']} - Warmup: {hyp['warmup_type']} - % local data: {hyp['local_data_percentage']} - Distill weight: {hyp['distill_weight']}"
        }
    )

    mean = base.mark_line().encode(
        x='round',
        y='mean(value)',
        color=alt.Color('line:N', scale=alt.Scale(range=['black']), legend=alt.Legend(symbolType="stroke"), title=""),
    )

    band = base.mark_errorband(extent='ci').encode(
        x='round',
        y=alt.Y('value', title='Accuracy on the test set'),
        color=alt.Color('shade:N', scale=alt.Scale(range=['steelblue']), legend=alt.Legend(symbolType="circle"), title=""),
    )



    plot = (mean + band).resolve_scale(
        color='independent'
    )
    return plot

In [4]:
generate_plot(data, list_of_experiments[14].hyperparameters)

alt.LayerChart(...)